# Ukara 1.0 Challenge Track 1

## Load Data

In [1]:
import pandas as pd

DIR_DATA_A = "../data/data_A"
DIR_DATA_B = "../data/data_B"
DIR_DATA_OUTPUT = '../data/submission'

In [2]:
data_A_train = pd.read_csv("{}/data_train_A.csv".format(DIR_DATA_A))
data_A_dev = pd.read_csv("{}/data_dev_A.csv".format(DIR_DATA_A))


data_B_train = pd.read_csv("{}/data_train_B.csv".format(DIR_DATA_B))
data_B_dev = pd.read_csv("{}/data_dev_B.csv".format(DIR_DATA_B))

In [3]:
print(data_A_train.shape)
data_A_train.head()

(268, 3)


,RES_ID,RESPONSE,LABEL
0,TRA1,intetraksi/beradaptasi terhadap lingkungan yan...,1
1,TRA2,seperti jatuhnya meteor tsunami gempa bumi,0
2,TRA3,hanya tuhan yang tahu tantangan nya itu apaan,0
3,TRA4,mereka akan sulit beradaptasi,1
4,TRA5,"Tempat tinggal, ekonomi, dan pekerjaan",1


In [4]:
print(data_A_dev.shape)
data_A_dev.head()

(215, 2)


,RES_ID,RESPONSE
0,DEVA1,karena di tempat tnggalnya tidak aman karena b...
1,DEVA2,"banyak yang menolak, susah mencari lapangan pe..."
2,DEVA3,ah berkomunikasi dengan wilyah yang mereka tem...
3,DEVA4,"tantangan yang akan mereka hadapi, seperti mer..."
4,DEVA5,Mereka akan tidak terbiasa dengan tempat baru


In [5]:
print(data_B_train.shape)
data_B_train.head()

(305, 3)


,RES_ID,RESPONSE,LABEL
0,TRB1,Karena orang berpikir bahwa jika disumbangkan ...,1
1,TRB2,Pakaian awet adalah alternatif terhadap Fast F...,0
2,TRB3,karna orang lebih suka menyumbang,1
3,TRB4,karana harga nya terjangkau dan pas.,0
4,TRB5,kerena harganya mahal .,0


In [6]:
print(data_B_dev.shape)
data_B_dev.head()

(244, 2)


,RES_ID,RESPONSE
0,DEVB1,karena jika menyumbangkan pakaian itu pakaian ...
1,DEVB2,memastikan bahwa ppenerima akan senang dan yg ...
2,DEVB3,karena banyak orang yang mendukung hal tersebu...
3,DEVB4,karena banayak orang yang ada atau bekerja di ...
4,DEVB5,karna sebagian orang ada yang ingin beramal da...


## Preprocess

In [7]:
import nltk

# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import stopwords
# import string
 
# kalimat = "[MOJOK.co] Manfaat jogging setiap pagi yang pertama adalah meredakan stres. Olahraga itu seperti kode bagi tubuh untuk memproduksi hormon endorfin, agen perangsang rasa bahagia. Dilakukan di pagi hari, ketika udara masih bersih, sejuk, jalanan lengang, gunung terlihat jelas di sebelah utara, manfaat jogging bisa kamu rasakan secara maksimal."
# kalimat = kalimat.translate(str.maketrans('','',string.punctuation)).lower()
 
# tokens = word_tokenize(kalimat)
# listStopword =  set(stopwords.words('indonesian'))
 
# removed = []
# for t in tokens:
#     if t not in listStopword:
#         removed.append(t)
 
# print(removed)
# # Hasil
# # ['mojokco', 'manfaat', 'jogging', 'pagi', 'meredakan', 'stres', 'olahraga', 'kode', 'tubuh', 'memproduksi', 'hormon', 'endorfin', 'agen', 'perangsang', 'bahagia', 'pagi', 'udara', 'bersih', 'sejuk', 'jalanan', 'lengang', 'gunung', 'sebelah', 'utara', 'manfaat', 'jogging', 'rasakan', 'maksimal']

In [8]:
import itertools
def normalizing_words(review):
    return ''.join(''.join(s)[:1] for _, s in itertools.groupby(review))

In [9]:
def preprocess(text):
    text = text.lower()
    text = nltk.tokenize.word_tokenize(text)
    text = ' '.join(text)
    text = normalizing_words(text)
    return text

data_A_train['RESPONSE'] = data_A_train['RESPONSE'].apply(lambda x: preprocess(x))
data_A_dev['RESPONSE'] = data_A_dev['RESPONSE'].apply(lambda x: preprocess(x))
data_B_train['RESPONSE'] = data_B_train['RESPONSE'].apply(lambda x: preprocess(x))
data_B_dev['RESPONSE'] = data_B_dev['RESPONSE'].apply(lambda x: preprocess(x))
data_A_train.head()

,RES_ID,RESPONSE,LABEL
0,TRA1,intetraksi/beradaptasi terhadap lingkungan yan...,1
1,TRA2,seperti jatuhnya meteor tsunami gempa bumi,0
2,TRA3,hanya tuhan yang tahu tantangan nya itu apan,0
3,TRA4,mereka akan sulit beradaptasi,1
4,TRA5,"tempat tingal , ekonomi , dan pekerjan",1


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer = "word",
                             tokenizer = None, 
                             preprocessor = None,
                             stop_words = None) 

data_A_train_feature = vectorizer.fit_transform(data_A_train['RESPONSE'].astype(str))
data_A_dev_feature = vectorizer.transform(data_A_dev['RESPONSE'].astype(str))

data_B_train_feature = vectorizer.fit_transform(data_B_train['RESPONSE'].astype(str))
data_B_dev_feature = vectorizer.transform(data_B_dev['RESPONSE'].astype(str))
data_A_train_feature[0]

<1x641 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [11]:
data_A_train_feature = data_A_train_feature.toarray()
data_A_dev_feature = data_A_dev_feature.toarray()
data_B_train_feature = data_B_train_feature.toarray()
data_B_dev_feature = data_B_dev_feature.toarray()
data_A_train_feature[0][0:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## Model

In [12]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from lightgbm import LGBMClassifier
import numpy as np

RANDOM_STATE = 1
def get_cv():
    return RepeatedStratifiedKFold(
        n_splits=5,
        n_repeats=10,
        random_state=RANDOM_STATE
    )

def print_proba_fp(y_true, proba, threshold):
    y_true = y_true.reset_index(drop=True)
    for i in range(len(y_true)):
        if (proba[i] >= threshold) and (y_true[i] == 0):
            print(proba[i])

## Model A

In [13]:
precision = []
recall = []
f1 = []

# best_params = {'subsample': 0.7711258797627427, 'colsample_bytree': 0.4005361499419192, 'num_leaves': 120, 'max_depth': 12, 'n_estimators': 200, 'learning_rate': 0.01385, 'reg_alpha': 0.17606142655245077, 'reg_lambda': 0.7877114465717386, 'random_state': 1}
best_params = {
    'random_state' : 1
}

X = data_A_train_feature
y = data_A_train['LABEL']
for train_index, test_index in get_cv().split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = LGBMClassifier(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:,1]
    threshold = 0.50
    
    y_pred = [1 if proba >= threshold else 0 for proba in y_pred]
    
    precision.append(precision_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    f1.append(f1_score(y_test, y_pred))
    
print("precision: %.5f (+/- %.2f)" % (np.mean(precision), np.std(precision)))
print("recall: %.5f (+/- %.2f)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.5f (+/- %.3f)" % (np.mean(f1), np.std(f1)))

precision: 0.83622 (+/- 0.04)
recall: 0.86267 (+/- 0.06)
f1 score: 0.84808 (+/- 0.034)


In [14]:
model = LGBMClassifier(**best_params)
model.fit(X, y)
y_pred_A = model.predict_proba(data_A_dev_feature)[:,1]
threshold = 0.50
y_pred_A = [1 if proba >= threshold else 0 for proba in y_pred_A]

print(y_pred_A[0:10])
print(len(y_pred_A))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
215


## Model B

In [15]:
precision = []
recall = []
f1 = []

# best_params = {'subsample': 0.6067495802697418, 'colsample_bytree': 0.4111886456872439, 'num_leaves': 112, 'max_depth': 7, 'n_estimators': 170, 'learning_rate': 0.00194, 'reg_alpha': 0.3506273661016787, 'reg_lambda': 0.26115409586224014, 'random_state': 1}
best_params = {
    'random_state' : 1
}

X = data_B_train_feature
y = data_B_train['LABEL']
for train_index, test_index in get_cv().split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = LGBMClassifier(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:,1]
    threshold = 0.50
    
    y_pred = [1 if proba >= threshold else 0 for proba in y_pred]
    
    precision.append(precision_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    f1.append(f1_score(y_test, y_pred))
    
print("precision: %.5f (+/- %.2f)" % (np.mean(precision), np.std(precision)))
print("recall: %.5f (+/- %.2f)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.5f (+/- %.3f)" % (np.mean(f1), np.std(f1)))

precision: 0.66967 (+/- 0.05)
recall: 0.66237 (+/- 0.08)
f1 score: 0.66377 (+/- 0.054)


In [16]:
model = LGBMClassifier(**best_params)
model.fit(X, y)
y_pred_B = model.predict_proba(data_B_dev_feature)[:,1]
threshold = 0.50
y_pred_B = [1 if proba >= threshold else 0 for proba in y_pred_B]

print(y_pred_B[0:10])

[1, 1, 1, 1, 0, 1, 0, 1, 1, 1]


## Save to JSON

In [17]:
prediksi_data_A = {'RES_ID': data_A_dev['RES_ID'],
                  'LABEL': y_pred_A
                  }

In [18]:
prediksi_data_B = {'RES_ID': data_B_dev['RES_ID'],
                   'LABEL': y_pred_B
                   }

In [19]:
df_final_A = pd.DataFrame(prediksi_data_A, columns= ['RES_ID', 'LABEL'])
df_final_B = pd.DataFrame(prediksi_data_B, columns= ['RES_ID', 'LABEL'])

In [20]:
df_final_B.head()

,RES_ID,LABEL
0,DEVB1,1
1,DEVB2,1
2,DEVB3,1
3,DEVB4,1
4,DEVB5,0


In [21]:
df_final = pd.concat([df_final_A, df_final_B])
df_final= df_final.reset_index(drop=True)
print(df_final.shape)
df_final.head()

(459, 2)


,RES_ID,LABEL
0,DEVA1,1
1,DEVA2,1
2,DEVA3,1
3,DEVA4,1
4,DEVA5,1


In [28]:
import datetime
currentDT = datetime.datetime.now()
df_final.to_json('{}/predictions_test_{}.json'.format(DIR_DATA_OUTPUT, currentDT), orient='records')